In [1]:
import numpy as np
import time

class gaussSolve:
    '''Class for solving linear equation with gaussian elimination'''
        
    def __init__(self,A0,b0,pivotTF=True,calcInv=True):
        self.A0=A0
        self.b0=b0
        self.N=len(b0)
        self.pivotTF=pivotTF
    
        Aandb=self.forward_elim(A0,b0)
        self.Aech=Aandb[0]
        self.bEch=Aandb[1]
        self.nPivots=Aandb[2]
        self.nSteps=Aandb[3]
        
        self.x=self.back_substitute(self.Aech,self.bEch)

        self.det=self.determinant(self.Aech,isEchelon=True,nPivots=self.nPivots)
        
        self.inv=self.inverse(A0) if calcInv else []   
       
        
    def __str__(self):
        
        return 'A original:\n'+str(self.A0)+'\n b orignial: '+str(self.b0) \
                +'\n A echelon: \n'+str(self.Aech)+'\n b echelon: '+str(self.bEch) \
                + '\n \n Determinant: ' + str(self.det) +'\n Inverse: \n'+str(self.inv) \
                +'\n \n x: '+str(self.x)
   
    def forward_elim(self,A0,b0):
        '''Perform forward elimination steps'''
        
        A=np.copy(A0)
        b=np.copy(b0)
        
        # TEST: lets see how many steps
        nSteps=0
        
        nPivots=0
        for column in range(self.N):
        
            if self.pivotTF:
                A,pivoted=self.pivot(A,b,column)
                if pivoted: nPivots+=1
            
            for row in range(column+1,self.N):              
                try:
                    multFac=(A[row,column]/A[column,column])
                except:
                    print("Issue with forward elimination!")
                A[row,:]-=multFac*A[column,:]
                b[row]-=multFac*b[column]
                
                nSteps+=1

        return A,b,nPivots,nSteps
            
    def back_substitute(self,A,b):
        '''Perform back substitution on row echelon matrix'''
        
        x=np.zeros(self.N)
        x[-1]=b[-1]/A[self.N-1,self.N-1]
        
        for row in range(self.N-2,-1,-1):
            for column in range(self.N-1,row,-1):
                x[row]-=x[column]*A[row,column]/A[row,row]
            x[row]+=b[row]/A[row,row]
            
        return x
            
    def pivot(self,A,b,column):
        '''Puts row with largest scaled value in column at row'''
        
        # Make sure we are only looking below row=column
        pivotRow=np.argmax((A[:,column]/np.max(np.abs(A), 1))[column:self.N])+column # Figure out if leading column has largest value
        A[[column,pivotRow]]=A[[pivotRow,column]] # If not, these lines will pivot
        b[[column,pivotRow]]=b[[pivotRow,column]]
        
        pivoted=True if column != pivotRow else False
        
        return A,pivoted

    def determinant(self,A,isEchelon=False,nPivots=-1):
        '''Calculate the determinant Gauss elimination'''
        
        if isEchelon: 
            if nPivots >= 0:
                return (-1.0)**nPivots*np.prod(np.diagonal(A))
            else:
                print('Must supply nPivots for determinant of Echelon matrix')
                raise
        
        bDummy=np.zeros(self.N)
        
        Aech,bDummy,nPivots=self.forward_elim(A,bDummy)
        
        return (-1.0)**nPivots*np.prod(np.diagonal(Aech))
    
    def inverse(self,A0):
        '''Calculate the inverse via Gauss elimination'''
        
        invA=np.identity(self.N)
        
        for column in range(self.N):
            A,e,nPivots,nSteps=self.forward_elim(A0,invA[:,column])
            x=self.back_substitute(A,e)
            
            invA[:,column]=x
        
        return invA
            

In [2]:
# Some examples:

#A=np.array([[1,1,1],[-1,2,0],[2,0,1]],float)
#=np.array([6.0,3.0,5.0])

eps=1e-15
A=np.array([[eps,1,1],[1,1,0],[1,0,1]],float)
b=np.array([5.0,3.0,4.0])

gaussTest=gaussSolve(A,b)
#gaussTest=gaussSolve(A,b,pivotTF=False)

print(gaussTest,'\n')
print('numpy determinant: ',np.linalg.det(A),'\n')
print('numpy inverse: \n',np.linalg.inv(A),'\n')
print('numpy solve x: ',np.linalg.solve(A,b),'\n')


A original:
[[1.e-15 1.e+00 1.e+00]
 [1.e+00 1.e+00 0.e+00]
 [1.e+00 0.e+00 1.e+00]]
 b orignial: [5. 3. 4.]
 A echelon: 
[[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 2.]]
 b echelon: [3. 5. 6.]
 
 Determinant: -1.999999999999999
 Inverse: 
[[-0.5  0.5  0.5]
 [ 0.5  0.5 -0.5]
 [ 0.5 -0.5  0.5]]
 
 x: [1. 2. 3.] 

numpy determinant:  -1.9999999999999991 

numpy inverse: 
 [[-0.5  0.5  0.5]
 [ 0.5  0.5 -0.5]
 [ 0.5 -0.5  0.5]] 

numpy solve x:  [1. 2. 3.] 



In [3]:
# Let's try a BIG random matrix
nMatrix=100
A=np.random.rand(nMatrix,nMatrix)
b=np.random.rand(nMatrix)

gaussTest=gaussSolve(A,b)

print(gaussTest.x,'\n')
print('numpy solve x: ',np.linalg.solve(A,b),'\n')
print(gaussTest.inv)

[ 2.28329189e-01  4.95144765e-01 -8.68321604e-01 -1.22022057e+00
  9.80377792e-01 -5.43142381e-01  1.06260542e+00 -1.15321653e+00
 -1.19708713e+00  6.67110158e-01  8.59131043e-02  6.33362481e-01
  2.99768436e-01 -2.65423687e-01 -1.01717634e+00  7.87761919e-01
 -4.83591021e-01 -7.18714884e-01  1.88389392e+00 -1.74852850e+00
  1.46987753e+00  5.43996878e-01  1.33123742e+00  6.06888095e-01
  5.74093758e-01  1.32101303e+00  2.32389192e-01 -1.24233919e+00
  1.57718847e+00  1.58834183e+00 -1.69513784e+00 -2.30684046e-01
 -6.17053256e-01 -7.31225205e-01 -5.86027647e-02 -6.59757240e-02
 -1.29172823e+00 -1.65905579e+00 -1.75716494e-04  2.26130489e-01
 -7.18295157e-01  7.98109783e-01  2.18295215e+00  7.26143228e-01
  1.42266359e+00 -1.60862585e+00 -4.52351561e-01 -6.40855545e-02
 -4.00442684e-01  1.99979835e-02  2.20069203e-01  1.01177097e+00
 -7.86146886e-01  1.11004120e-01  3.22371947e-01 -7.33481758e-01
  6.96114495e-02  5.97419432e-01  4.30814009e-01  1.23741925e-01
  1.17833592e-01 -7.42757

In [7]:
# Example of a singular matrix:

A = np.array([ [ 1+1, 2, 3], 
               [ 4, 5, 6], 
               [ 7, 8, 9] ], dtype=np.float64)
b = np.array([5, -1, 2], dtype=np.float64)

gaussSing=gaussSolve(A,b)

print(gaussSing)

print('Condition number: ',np.linalg.cond(A))

A original:
[[2. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
 b orignial: [ 5. -1.  2.]
 A echelon: 
[[7.         8.         9.        ]
 [0.         0.42857143 0.85714286]
 [0.         0.         1.        ]]
 b echelon: [ 2.         -2.14285714  3.        ]
 
 Determinant: -3.0000000000000018
 Inverse: 
[[ 1.         -2.          1.        ]
 [-2.          1.          0.        ]
 [ 1.          0.66666667 -0.66666667]]
 
 x: [  9. -11.   3.]
Condition number:  52.362349349617176


In [8]:
# Banded matricies

class gaussSolveBanded(gaussSolve):
    
    def __init__(self,A0,b0,nBands,pivotTF=True,calcInv=False):
        
        self.nBands=nBands
        gaussSolve.__init__(self,A0,b0,pivotTF=True,calcInv=False)

    def forward_elim(self,A0,b0):
        '''Perform forward elimination steps for banded matrix'''
        
        A=np.copy(A0)
        b=np.copy(b0)
        
        # TEST: lets see how many steps
        nSteps=0
        
        nPivots=0 # No pivots, or we lose the banded nature
        for column in range(self.N):
            for band in range(column+1,min(self.N,column+1+self.nBands)):
                try:
                    multFac=(A[band,column]/A[column,column])
                except:
                    print(band,column)
                    print("Issue with forward elimination!")
                    
                A[band,:]-=multFac*A[column,:]
                b[band]-=multFac*b[column]

                nSteps+=1
                
        return A,b,nPivots,nSteps
    


In [10]:
# Construct random banded matrix
nBands=3
nMatrix=100
A=np.random.rand(nMatrix,nMatrix)
b=np.random.rand(nMatrix)

for ii in range(nMatrix):
    for jj in range(nMatrix):
        if abs(ii-jj)>nBands: A[ii,jj]=0.0
            
np.set_printoptions(linewidth=100)

# Time the normal algorithm versus the banded one
start = time.perf_counter()
gaussBand=gaussSolve(A,b,calcInv=False)
end = time.perf_counter()
print("Time for gaussSolve:", end - start)
print(gaussBand.x)

print()

start = time.perf_counter()
gaussBand2=gaussSolveBanded(A,b,nBands,calcInv=False)
end = time.perf_counter()
print("Time for gaussSolveBanded:", end - start)
print(gaussBand2.x)


Time for gaussSolve: 0.03737359499791637
[-0.9332891   1.3529344  -0.36662614  0.23695011  1.22341427 -0.1385513   0.95144775 -1.17341228
 -0.58288555  0.23564709 -1.3900759   2.36630449  0.71886905  1.72702869  0.48170646 -0.65170698
 -2.58669817 -1.36103324  0.45299263 -0.23945091  9.74951177 -4.70915011 -3.10869439  4.52104931
 -9.98127024  4.34879253  1.56373923  3.88324126  3.57050443 -5.85333275  2.76354692 -0.29041671
  0.52869166  2.1854948  -2.06004048 -1.06511919  3.48422163  0.20064843  0.3316508  -0.19470155
  5.45155599 -1.38889873  0.68501818 -4.08425607 -0.80426384  1.88535645  0.2164382   1.25506549
  2.80441892 -1.02821892 -1.77274256 -2.26053005  4.29189817  1.17874119 -0.52057439 -0.09508151
  0.0104386  -2.49210283  1.14384793  1.08995091  1.69465464 -0.39141846 -0.01490338  1.07544379
 -2.3995114  -4.45671769  3.70380329 -0.62460146  5.69294528 -2.47039582  1.22371174 -3.6692216
  5.71345648 -3.11078559 -0.66750993  2.92407335  0.33641634  4.27442772  0.56042419 -2